In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from model_conv1d import CNN1D, TrainingHarness
from preprocessing import create_tensor, standard_scaling, cv_split, create_tensor
from utils import save_pred

%load_ext autoreload
%autoreload 2
%aimport numpy
%aimport torch
%aimport matplotlib.pyplot

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_df = pd.read_csv("./data/train.csv", index_col=0).drop_duplicates()
test_df = pd.read_csv("./data/test_nolabels.csv", index_col=0)

In [ ]:
scaled_train_df, scaled_test_df = standard_scaling(train_df, test_df)

In [ ]:
def create_torch_dataset(train_df):
    sets = cv_split(train_df, n_splits=2)

    x_train = torch.as_tensor(create_tensor(sets[0]["train"][0]), dtype=torch.float).to(device)
    y_train = torch.as_tensor(sets[0]["train"][1].to_numpy(), dtype=torch.float).to(device)

    x_val = torch.as_tensor(create_tensor(sets[0]["test"][0]), dtype=torch.float).to(device)
    y_val = torch.as_tensor(sets[0]["test"][1].to_numpy(), dtype=torch.float).to(device)

    return ((x_train, y_train), (x_val, y_val))

In [ ]:
dataset = create_torch_dataset(scaled_train_df)

In [ ]:
model_CNN1d = CNN1D()
model_CNN1d.to(device)

In [ ]:
harness = TrainingHarness(dataset=dataset, model=model_CNN1d)

In [ ]:
logs = harness.train_loop(200)

In [ ]:
harness.plot_logs()

In [ ]:
x_test = torch.as_tensor(create_tensor(scaled_test_df), dtype=torch.float).to(device)
bin_pred = harness.model(x_test).argmax(axis=1).detach().numpy()

In [ ]:
output_df = pd.DataFrame(bin_pred, index=test_df.index, columns=["LABELS"], dtype=int)
output_df.to_csv(f"./submission/submission8.csv")

In [ ]:
save_pred(test_df, bin_pred, 8)